In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('attribution_allocation_student_data.csv')
df

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier
0,True,referral,referral,referral,email,NaN,1
1,True,referral,display,display,email,NaN,1
2,True,email,display,referral,NaN,NaN,1
3,True,referral,referral,email,NaN,NaN,1
4,True,social,referral,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...
68119,False,social,display,direct,NaN,NaN,3
68120,False,referral,referral,referral,referral,direct,3
68121,False,display,referral,social,direct,NaN,3
68122,False,email,email,social,direct,NaN,3


## Method 1: Last interaction

In [3]:
tier1 = df[df['tier']==1]
tier2 = df[df['tier']==2]
tier3 = df[df['tier']==3]
tier1_con = tier1[tier1['convert_TF'] == True ]
tier2_con = tier2[tier2['convert_TF'] == True ]
tier3_con = tier3[tier3['convert_TF'] == True ]
##social, organic_search, referral, email, paid_search, display, direct
li = ['social', 'organic_search', 'referral', 'email', 'paid_search', 'display', 'direct']
cnt1 = dict.fromkeys(li, 0)
cnt2 = dict.fromkeys(li, 0)
cnt3 = dict.fromkeys(li, 0)

for i in range(len(tier1_con)):
    if not tier1_con.iloc[i].isna()['touch5']:
        cnt1[tier1_con.iloc[i]['touch5']]+=1
    else:
        attr_id = list(tier1_con.iloc[i].isna()).index(True)-1
        cnt1[tier1_con.iloc[i][attr_id]]+=1
        
for i in range(len(tier2_con)):
    if not tier2_con.iloc[i].isna()['touch5']:
        cnt2[tier2_con.iloc[i]['touch5']]+=1
    else:
        attr_id = list(tier2_con.iloc[i].isna()).index(True)-1
        cnt2[tier2_con.iloc[i][attr_id]]+=1
        
for i in range(len(tier3_con)):
    if not tier3_con.iloc[i].isna()['touch5']:
        cnt3[tier3_con.iloc[i]['touch5']]+=1
    else:
        attr_id = list(tier3_con.iloc[i].isna()).index(True)-1
        cnt3[tier3_con.iloc[i][attr_id]]+=1

In [4]:
###Calculate average CAC
cnt_m1 = pd.concat([pd.DataFrame.from_dict(cnt1, orient='index'),
       pd.DataFrame.from_dict(cnt2, orient='index'),
       pd.DataFrame.from_dict(cnt3, orient='index')], axis=1)
cnt_m1.columns=['tier1','tier2','tier3']
cnt_m1['total'] = cnt_m1['tier1']+cnt_m1['tier2']+cnt_m1['tier3']
# cnt_m1['tier1/total_%'] = round(cnt_m1['tier1']/cnt_m1['total'] * 100, 2)
# cnt_m1['tier2/total_%'] = round(cnt_m1['tier2']/cnt_m1['total'] * 100, 2)
# cnt_m1['tier3/total_%'] = round(cnt_m1['tier3']/cnt_m1['total'] * 100, 2)
cnt_m1['tier1/tier3_%'] = round(cnt_m1['tier1']/cnt_m1['tier3'] * 100, 2)
cnt_m1['tier2/tier3_%'] = round(cnt_m1['tier2']/cnt_m1['tier3'] * 100, 2)
cnt_m1

,tier1,tier2,tier3,total,tier1/tier3_%,tier2/tier3_%
social,625,1060,1275,2960,49.02,83.14
organic_search,2,4,5,11,40.00,80.00
referral,1211,2107,2696,6014,44.92,78.15
email,281,555,655,1491,42.90,84.73
paid_search,176,347,481,1004,36.59,72.14
display,528,818,946,2292,55.81,86.47
direct,1,1,5,7,20.00,20.00


In [5]:
cnt_m1['AVG_CAC'] = (1000+2000+3000)/cnt_m1['total']
cnt_m1.drop(index='organic_search',inplace=True)
cnt_m1.drop(index='direct',inplace=True)
cnt_m1

,tier1,tier2,tier3,total,tier1/tier3_%,tier2/tier3_%,AVG_CAC
social,625,1060,1275,2960,49.02,83.14,2.027027
referral,1211,2107,2696,6014,44.92,78.15,0.997672
email,281,555,655,1491,42.90,84.73,4.024145
paid_search,176,347,481,1004,36.59,72.14,5.976096
display,528,818,946,2292,55.81,86.47,2.617801


## Method 2: Last non-direct click

In [6]:
li = ['social', 'organic_search', 'referral', 'email', 'paid_search', 'display', 'direct', True]
cnt1 = dict.fromkeys(li, 0)
cnt2 = dict.fromkeys(li, 0)
cnt3 = dict.fromkeys(li, 0)

for i in range(len(tier1_con)):
    if not tier1_con.iloc[i].isna()['touch5']:
        if tier1_con.iloc[i]['touch5'] =='direct':
            cnt1[tier1_con.iloc[i]['touch4']]+=1
        else:
            cnt1[tier1_con.iloc[i]['touch5']]+=1
    else:
        attr_id = list(tier1_con.iloc[i].isna()).index(True)-1
        if tier1_con.iloc[i][attr_id] =='direct':
            cnt1[tier1_con.iloc[i][attr_id-1]]+=1
        else:
            cnt1[tier1_con.iloc[i][attr_id]]+=1
        
for i in range(len(tier2_con)):
    if not tier2_con.iloc[i].isna()['touch5']:
        if tier2_con.iloc[i]['touch5'] =='direct':
            cnt2[tier2_con.iloc[i]['touch4']]+=1
        else:
            cnt2[tier2_con.iloc[i]['touch5']]+=1
    else:
        attr_id = list(tier2_con.iloc[i].isna()).index(True)-1
        if tier2_con.iloc[i][attr_id] =='direct':
            cnt2[tier2_con.iloc[i][attr_id-1]]+=1
        else:
            cnt2[tier2_con.iloc[i][attr_id]]+=1
            
for i in range(len(tier3_con)):
    if not tier3_con.iloc[i].isna()['touch5']:
        if tier3_con.iloc[i]['touch5'] =='direct':
            cnt3[tier3_con.iloc[i]['touch4']]+=1
        else:
            cnt3[tier3_con.iloc[i]['touch5']]+=1
    else:
        attr_id = list(tier3_con.iloc[i].isna()).index(True)-1
        if tier3_con.iloc[i][attr_id] =='direct':
            cnt3[tier3_con.iloc[i][attr_id-1]]+=1
        else:
            cnt3[tier3_con.iloc[i][attr_id]]+=1

In [7]:
###Calculate average CAC
cnt_m2 = pd.concat([pd.DataFrame.from_dict(cnt1, orient='index'),
       pd.DataFrame.from_dict(cnt2, orient='index'),
       pd.DataFrame.from_dict(cnt3, orient='index')], axis=1)
cnt_m2.columns=['tier1','tier2','tier3']
cnt_m2.drop(index=True,inplace=True)
cnt_m2['total'] = cnt_m2['tier1']+cnt_m2['tier2']+cnt_m2['tier3']
# cnt_m2['tier1/total_%'] = round(cnt_m2['tier1']/cnt_m2['total'] * 100, 2)
# cnt_m2['tier2/total_%'] = round(cnt_m2['tier2']/cnt_m2['total'] * 100, 2)
# cnt_m2['tier3/total_%'] = round(cnt_m2['tier3']/cnt_m2['total'] * 100, 2)
cnt_m2['tier1/tier3_%'] = round(cnt_m2['tier1']/cnt_m2['tier3'] * 100, 2)
cnt_m2['tier2/tier3_%'] = round(cnt_m2['tier2']/cnt_m2['tier3'] * 100, 2)
cnt_m2

,tier1,tier2,tier3,total,tier1/tier3_%,tier2/tier3_%
social,625,1061,1275,2961,49.02,83.22
organic_search,2,4,5,11,40.00,80.00
referral,1212,2107,2698,6017,44.92,78.09
email,281,555,656,1492,42.84,84.60
paid_search,176,347,481,1004,36.59,72.14
display,528,818,947,2293,55.76,86.38
direct,0,0,0,0,NaN,NaN


In [8]:
cnt_m2['AVG_CAC'] = (1000+2000+3000)/cnt_m2['total']
cnt_m2.drop(index='organic_search',inplace=True)
cnt_m2.drop(index='direct',inplace=True)
cnt_m2

,tier1,tier2,tier3,total,tier1/tier3_%,tier2/tier3_%,AVG_CAC
social,625,1061,1275,2961,49.02,83.22,2.026342
referral,1212,2107,2698,6017,44.92,78.09,0.997175
email,281,555,656,1492,42.84,84.60,4.021448
paid_search,176,347,481,1004,36.59,72.14,5.976096
display,528,818,947,2293,55.76,86.38,2.616659


## Method 3: First interaction

In [9]:
li = ['social', 'organic_search', 'referral', 'email', 'paid_search', 'display', 'direct']
cnt1 = dict.fromkeys(li, 0)
cnt2 = dict.fromkeys(li, 0)
cnt3 = dict.fromkeys(li, 0)

for i in range(len(tier1_con)):
    cnt1[tier1_con.iloc[i]['touch1']]+=1
        
for i in range(len(tier2_con)):
    cnt2[tier2_con.iloc[i]['touch1']]+=1

for i in range(len(tier3_con)):
    cnt3[tier3_con.iloc[i]['touch1']]+=1    

In [10]:
###Calculate average CAC
cnt_m3 = pd.concat([pd.DataFrame.from_dict(cnt1, orient='index'),
       pd.DataFrame.from_dict(cnt2, orient='index'),
       pd.DataFrame.from_dict(cnt3, orient='index')], axis=1)
cnt_m3.columns=['tier1','tier2','tier3']
cnt_m3['total'] = cnt_m3['tier1']+cnt_m3['tier2']+cnt_m3['tier3']
# cnt_m3['tier1/total_%'] = round(cnt_m3['tier1']/cnt_m3['total'] * 100, 2)
# cnt_m3['tier2/total_%'] = round(cnt_m3['tier2']/cnt_m3['total'] * 100, 2)
# cnt_m3['tier3/total_%'] = round(cnt_m3['tier3']/cnt_m3['total'] * 100, 2)
cnt_m3['tier1/tier3_%'] = round(cnt_m3['tier1']/cnt_m3['tier3'] * 100, 2)
cnt_m3['tier2/tier3_%'] = round(cnt_m3['tier2']/cnt_m3['tier3'] * 100, 2)
cnt_m3

,tier1,tier2,tier3,total,tier1/tier3_%,tier2/tier3_%
social,475,823,982,2280,48.37,83.81
organic_search,1,7,7,15,14.29,100.00
referral,1519,2719,3257,7495,46.64,83.48
email,264,402,618,1284,42.72,65.05
paid_search,129,211,328,668,39.33,64.33
display,435,726,870,2031,50.00,83.45
direct,1,4,1,6,100.00,400.00


In [11]:
cnt_m3['AVG_CAC'] = (1000+2000+3000)/cnt_m3['total']
cnt_m3.drop(index='organic_search',inplace=True)
cnt_m3.drop(index='direct',inplace=True)
cnt_m3

,tier1,tier2,tier3,total,tier1/tier3_%,tier2/tier3_%,AVG_CAC
social,475,823,982,2280,48.37,83.81,2.631579
referral,1519,2719,3257,7495,46.64,83.48,0.800534
email,264,402,618,1284,42.72,65.05,4.672897
paid_search,129,211,328,668,39.33,64.33,8.982036
display,435,726,870,2031,50.00,83.45,2.954210


## Linear

In [12]:
li = ['social', 'organic_search', 'referral', 'email', 'paid_search', 'display', 'direct']
cnt1 = dict.fromkeys(li, 0)
cnt2 = dict.fromkeys(li, 0)
cnt3 = dict.fromkeys(li, 0)

for i in range(len(tier1_con)):
    counter = 0
    weight = -1
    for n in range(5):
        if tier1_con.iloc[i]['touch'+str(n+1)] in li:
            counter+=1
    weight = 1/counter
#    print(weight)
    for n in range(5):
        if tier1_con.iloc[i]['touch'+str(n+1)] in li:
            cnt1[tier1_con.iloc[i]['touch'+str(n+1)]] += weight
#    print(cnt1)
        
for i in range(len(tier2_con)):
    counter = 0
    weight = -1
    for n in range(5):
        if tier2_con.iloc[i]['touch'+str(n+1)] in li:
            counter+=1
    weight = 1/counter
    for n in range(5):      
        if tier2_con.iloc[i]['touch'+str(n+1)] in li:
            #cnt2[tier2_con.iloc[i]['touch1']] += weight
            cnt2[tier2_con.iloc[i]['touch'+str(n+1)]] += weight
# print(cnt2)
            
for i in range(len(tier3_con)):
    counter = 0
    weight = -1
    for n in range(5):
        if tier3_con.iloc[i]['touch'+str(n+1)] in li:
            counter+=1
    weight = 1/counter
    for n in range(5):
        if tier3_con.iloc[i]['touch'+str(n+1)] in li:
            cnt3[tier3_con.iloc[i]['touch'+str(n+1)]] += weight
# print(cnt3)
        

In [13]:
###Calculate average CAC
cnt_m4 = pd.concat([pd.DataFrame.from_dict(cnt1, orient='index'),
       pd.DataFrame.from_dict(cnt2, orient='index'),
       pd.DataFrame.from_dict(cnt3, orient='index')], axis=1)
cnt_m4.columns=['tier1','tier2','tier3']
cnt_m4['total'] = cnt_m4['tier1']+cnt_m4['tier2']+cnt_m4['tier3']
cnt_m4['tier1/tier3_%'] = round(cnt_m4['tier1']/cnt_m4['tier3'] * 100, 2)
cnt_m4['tier2/tier3_%'] = round(cnt_m4['tier2']/cnt_m4['tier3'] * 100, 2)
cnt_m4

,tier1,tier2,tier3,total,tier1/tier3_%,tier2/tier3_%
social,484.583333,822.366667,995.233333,2302.183333,48.69,82.63
organic_search,1.950000,6.466667,7.733333,16.150000,25.22,83.62
referral,1517.100000,2656.683333,3290.050000,7463.833333,46.11,80.75
email,251.133333,442.516667,581.450000,1275.100000,43.19,76.11
paid_search,128.050000,237.816667,339.000000,704.866667,37.77,70.15
display,439.266667,724.083333,846.116667,2009.466667,51.92,85.58
direct,1.916667,2.066667,3.416667,7.400000,56.10,60.49


In [14]:
cnt_m4['AVG_CAC'] = (1000+2000+3000)/cnt_m4['total']
cnt_m4.drop(index='organic_search',inplace=True)
cnt_m4.drop(index='direct',inplace=True)
cnt_m4

,tier1,tier2,tier3,total,tier1/tier3_%,tier2/tier3_%,AVG_CAC
social,484.583333,822.366667,995.233333,2302.183333,48.69,82.63,2.606222
referral,1517.100000,2656.683333,3290.050000,7463.833333,46.11,80.75,0.803876
email,251.133333,442.516667,581.450000,1275.100000,43.19,76.11,4.705513
paid_search,128.050000,237.816667,339.000000,704.866667,37.77,70.15,8.512248
display,439.266667,724.083333,846.116667,2009.466667,51.92,85.58,2.985867


# Part 2 Allocation

In [15]:
# Use Last interaction Method
######################
cnt_m1

,tier1,tier2,tier3,total,tier1/tier3_%,tier2/tier3_%,AVG_CAC
social,625,1060,1275,2960,49.02,83.14,2.027027
referral,1211,2107,2696,6014,44.92,78.15,0.997672
email,281,555,655,1491,42.90,84.73,4.024145
paid_search,176,347,481,1004,36.59,72.14,5.976096
display,528,818,946,2292,55.81,86.47,2.617801


In [16]:
# Calculate Marginal CAC
cnt_m1['0_1000_CAC'] = 1000/cnt_m1['tier1']
cnt_m1['1000_2000_CAC'] = 1000/(cnt_m1['tier2']-cnt_m1['tier1'])
cnt_m1['2000_3000_CAC'] = 1000/(cnt_m1['tier3']-cnt_m1['tier2'])
cnt_m1

,tier1,tier2,tier3,total,tier1/tier3_%,tier2/tier3_%,AVG_CAC,0_1000_CAC,1000_2000_CAC,2000_3000_CAC
social,625,1060,1275,2960,49.02,83.14,2.027027,1.600000,2.298851,4.651163
referral,1211,2107,2696,6014,44.92,78.15,0.997672,0.825764,1.116071,1.697793
email,281,555,655,1491,42.90,84.73,4.024145,3.558719,3.649635,10.000000
paid_search,176,347,481,1004,36.59,72.14,5.976096,5.681818,5.847953,7.462687
display,528,818,946,2292,55.81,86.47,2.617801,1.893939,3.448276,7.812500


In [17]:
#print('tier1_total:', cnt_m1['tier1'].sum(),'tier2_total:', cnt_m1['tier2'].sum(),'tier3_total:',cnt_m1['tier3'].sum(),'total:',cnt_m1['total'].sum())
#print('tier1_CAC', cnt_m1['tier1'].sum(),'tier2_CAC', cnt_m1['tier2'].sum(),'tier3_CAC',cnt_m1['tier3'].sum(),'total',cnt_m1['total_CAC'].sum())
dict_total = {'tier1_total':cnt_m1['tier1'].sum(),'tier2_total':cnt_m1['tier2'].sum(),'tier3_total':cnt_m1['tier3'].sum(),'total':cnt_m1['total'].sum(),'AVG_CAC(avg_for_all_channel)':cnt_m1['AVG_CAC'].mean(),'0_1000_CAC(avg_for_all_channel)':cnt_m1['0_1000_CAC'].mean(),'1000_2000_CAC(avg_for_all_channel)':cnt_m1['1000_2000_CAC'].mean(),'2000_3000_CAC(avg_for_all_channel)':cnt_m1['2000_3000_CAC'].mean() }   
dict_total   

{'tier1_total': 2821,
 'tier2_total': 4887,
 'tier3_total': 6053,
 'total': 13761,
 'AVG_CAC(avg_for_all_channel)': 3.1285481318659194,
 '0_1000_CAC(avg_for_all_channel)': 2.712048053702344,
 '1000_2000_CAC(avg_for_all_channel)': 3.2721572236447316,
 '2000_3000_CAC(avg_for_all_channel)': 6.324828445426361}

In [18]:
cnt_m1.loc['social/total_%',0:4] = [round(cnt_m1.loc['social','tier1']/2821*100,2), round(cnt_m1.loc['social','tier2']/4837*100,2), round(cnt_m1.loc['social','tier3']/6053*100,2), round(cnt_m1.loc['social','total']/13761*100,2)]
cnt_m1.loc['referral/total_%',0:4] = [round(cnt_m1.loc['referral','tier1']/2821*100,2), round(cnt_m1.loc['referral','tier2']/4837*100,2), round(cnt_m1.loc['referral','tier3']/6053*100,2), round(cnt_m1.loc['referral','total']/13761*100,2)]
cnt_m1.loc['email/total_%',0:4] = [round(cnt_m1.loc['email','tier1']/2821*100,2), round(cnt_m1.loc['email','tier2']/4837*100,2), round(cnt_m1.loc['email','tier3']/6053*100,2), round(cnt_m1.loc['email','total']/13761*100,2)]
cnt_m1.loc['paid_search/total_%',0:4] = [round(cnt_m1.loc['paid_search','tier1']/2821*100,2), round(cnt_m1.loc['paid_search','tier2']/4837*100,2), round(cnt_m1.loc['paid_search','tier3']/6053*100,2), round(cnt_m1.loc['paid_search','total']/13761*100,2)]
cnt_m1.loc['display/total_%',0:4] = [round(cnt_m1.loc['display','tier1']/2821*100,2), round(cnt_m1.loc['display','tier2']/4837*100,2), round(cnt_m1.loc['display','tier3']/6053*100,2), round(cnt_m1.loc['display','total']/13761*100,2)]
cnt_m1

<ipython-input-18-71d5b2e9e646>:1: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  cnt_m1.loc['social/total_%',0:4] = [round(cnt_m1.loc['social','tier1']/2821*100,2), round(cnt_m1.loc['social','tier2']/4837*100,2), round(cnt_m1.loc['social','tier3']/6053*100,2), round(cnt_m1.loc['social','total']/13761*100,2)]
<ipython-input-18-71d5b2e9e646>:2: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  cnt_m1.loc['referral/total_%',0:4] = [round(cnt_m1.loc['referral','tier1']/2821*100,2), round(cnt_m1.loc['referral','tier2']/4837*100,2), round(cnt_m1.loc['referral','tier3']/6053*100,2), round(cnt_m1.loc['referral','total']/13761*100,2)]
<ipython-input-18-71d5b2e9e646>:3: FutureWarning: Slicing a positional slice with .loc is not supported, and will ra

,tier1,tier2,tier3,total,tier1/tier3_%,tier2/tier3_%,AVG_CAC,0_1000_CAC,1000_2000_CAC,2000_3000_CAC
social,625.00,1060.00,1275.00,2960.00,49.02,83.14,2.027027,1.600000,2.298851,4.651163
referral,1211.00,2107.00,2696.00,6014.00,44.92,78.15,0.997672,0.825764,1.116071,1.697793
email,281.00,555.00,655.00,1491.00,42.90,84.73,4.024145,3.558719,3.649635,10.000000
paid_search,176.00,347.00,481.00,1004.00,36.59,72.14,5.976096,5.681818,5.847953,7.462687
display,528.00,818.00,946.00,2292.00,55.81,86.47,2.617801,1.893939,3.448276,7.812500
social/total_%,22.16,21.91,21.06,21.51,NaN,NaN,NaN,NaN,NaN,NaN
referral/total_%,42.93,43.56,44.54,43.70,NaN,NaN,NaN,NaN,NaN,NaN
email/total_%,9.96,11.47,10.82,10.83,NaN,NaN,NaN,NaN,NaN,NaN
paid_search/total_%,6.24,7.17,7.95,7.30,NaN,NaN,NaN,NaN,NaN,NaN
display/total_%,18.72,16.91,15.63,16.66,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
lis = pd.concat([cnt_m1['0_1000_CAC'],cnt_m1['1000_2000_CAC'],cnt_m1['2000_3000_CAC']])
lis = lis.reset_index()
lis.sort_values(0,inplace=True)
lis = lis.reset_index()

In [20]:
lis.iloc[:11]

,level_0,index,0
0,1,referral,0.825764
1,11,referral,1.116071
2,0,social,1.600000
3,21,referral,1.697793
4,4,display,1.893939
5,10,social,2.298851
6,14,display,3.448276
7,2,email,3.558719
8,12,email,3.649635
9,20,social,4.651163
